# Creating a Collaborative Closing Transaction

In this section, we'll build a Lightning channel closing transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed, signed, and the messages exchanged between peers to coordinate the close. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use the Chapter 1 – Channel Funding Transaction, where the funding transaction was created and confirmed on our regtest blockchain.

In [1]:
# run notebook
%run "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop/chapter 1 - channel funding transaction/funding-transaction.ipynb"

2025-08-30T00:36:01.171000Z TestFramework (INFO): PRNG seed is: 5423168156992911971
2025-08-30T00:36:01.172000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_o_s0hjo0
Alice funding pubkey: 150e1c6f6c0469d45bcc3dc200674edf6817e7eb33686e471b05b84899b8326f
Alice funding privkey: b0c9d682325bebdbd7ab0f0bb2e9ab6ed844b299c8431a5238dbf0bddd92cf46
Alice funding address: bcrt1pz58pcmmvq35agk7v8hpqqe6wma5p0eltxd5xu3cmqkuy3xdcxfhs5s7vpl
Bob funding pubkey: 056f58d227cd81e34d60fa09e4043249b418193e1b9a8e4ec98f99ac5a0f9a04
Bob funding privkey: 3e35c91e86c96df7251e2deabac16866d6074f7e1da7c8be4af5906e4d4dd673
Bob funding address: bcrt1pq4h43538ekq7xntqlgy7gppjfx6psxf7rwdgunkf37v6cks0ngzqrekcq6
Transaction creating Alice UTXO: edf8eb3e7368b89283fb3a8d0a3f3a3c58f242dbcf1f62f0d78d7ad604d93856
{
  "txid": "edf8eb3e7368b89283fb3a8d0a3f3a3c58f242dbcf1f62f0d78d7ad604d93856",
  "hash": "f46b3bee2a31b35fac5efe668bab2735e26b778f6f626f0aeeb38a53dc50aa23",
  "version": 2,
  "size": 289,

### Legacy cooperative closure

Compared to the base segwit v0 channel type, for simple taproot channels, the co-op close transaction now always signals RBF. In other words, the sequence field of the sole input to the cooperative close transaction MUST be less-than-or-equal to 0xfffffffd. This enables a future cooperative closure flow to support increasing the fee of subsequent close offers via RBF.

In addition, rather than adopt the existing cooperative closure fee rate "negotiation", the responder SHOULD now always accept the offer sent by the initiator. In other words, the cooperative close process now terminates after exactly 1 RTTs: initiator sends sigs with offer, with the responder echo'ing back the same fee rate. This serves to ensure that the co-op close process always terminates deterministically, and also plays nicer with the nonces: only a single message is ever signed by both sides for a coop close workflow.

    +-----------+                                        +---------+
    |           |--(1)-------- shutdown_nonce ---------->|         |
    |           |<-(2)-------- shutdown_nonce -----------|         |
    |           |                                        |         |
    |   Alice   |--(3)-- closing_signed (partial_sig) -->|   Bob   |
    |           |<-(4)-- closing_signed (partial_sig) ---|         |
    |           |                                        |         |
    |           |                                        |         |
    |           |                                        |         |
    +-----------+                                        +---------+

### Modern RBF-based cooperative close

For taproot channels supporting RBF cooperative close, nonces are delivered just-in-time (JIT) with signatures using an asymmetric pattern:

Initial Exchange: Nonces are exchanged in shutdown messages

Each party sends their "closee nonce" - the nonce they'll use when signing as the closee. This nonce is used by the remote party when they act as closer

#### RBF Iterations: Subsequent nonces are delivered with signatures

* closing_complete (from closer): uses partial_sig_w_nonce bundles the partial signature with the sender's closer nonce. The bundling is necessary because the closee doesn't know this nonce yet.

* closing_sig (from closee): uses partial_sig plus nonce (NextCloseeNonce) field

Asymmetric Roles:

* Closer: The party sending closing_complete, proposing a new fee
* Closee: The party sending closing_sig, accepting the fee proposal

Each party alternates between these roles during RBF iterations.

    +-----------+                                                +---------+
    |           |--(1)------------- shutdown_nonce ------------->|         |
    |           |<-(2)------------- shutdown_nonce --------------|         |
    |           |                                                |         |
    |   Alice   |--(3)- closing_complete (partial_sig_w_nonce) ->|   Bob   |
    |           |<-(4)---- closing_sig (partial_sig) (nonce)-----|         |
    |           |                                                |         |
    |           |<-(5)- closing_complete (partial_sig_w_nonce) ->|         |
    |           |--(6)---- closing_sig (partial_sig) (nonce)---->|         |
    |           |                                                |         |
    +-----------+                                                +---------+

